In [5]:
from datasets import Dataset
from config import evaluation_params_all
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = evaluation_params_all.similar_is_equal_threshold_list

### MISTRAL NoQuant FT

In [3]:
import glob
import pandas as pd
from datasets import Dataset
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = [100]
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/noQuant/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
output_cleaner = OutputCleaner(verbose=False)

print(evaluation_results)
for file in csv_files:
    if file.strip().endswith('0.0008.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal_threshold in similar_is_equal_threshold_list:
        # print(f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}")
        evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
        try:
            evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                                words_level=True, 
                                                similarity_types=['case', 'stop_words', 'subset', 'superset'])
            evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
        except Exception as e:
            print(e)
            break
        #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
        # print('DONE')

Empty DataFrame
Columns: [file, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/mistral/noQuant/maxNewTokensFactor2_nShotsInference0_mistral-7b-instruct-v0.2_adapters_en.layer1_NoQuant_torch.bfloat16_32_32_0.05_4_0.0002.csv
Extra data: line 1 column 281 (char 280)
FILE:  data/mistral/noQuant/maxNewTokensFactor2_nShotsInference0_mistral-7b-instruct-v0.2_adapters_en.layer1_NoQuant_torch.bfloat16_16_32_0.01_2_0.0002.csv
Extra data: line 1 column 281 (char 280)
FILE:  data/mistral/noQuant/maxNewTokensFactor4_nShotsInference0_mistral-7b-instruct-v0.2_adapters_en.layer1_NoQuant_torch.bfloat16_32_32_0.05_2_0.0002.csv
Extra data: line 1 column 281 (char 280)
FILE:  data/mistral/noQuant/maxNewTokensFactor8_nShotsInference0_mistral-7b-instruct-v0.2_adapters_en.layer1_NoQuant_torch.bfloat16_16_32_0.01_8_0.0002.csv
Extra data: line 1 column 281 (char 280)
FILE:  data/mistral/noQuant/maxNewTokensFactor8_nShotsInference0_mistral-7b-instruct-v0.2_adapters_en.layer1_NoQu

In [6]:
cleaned_data[5]

{'sentence': 'A 50-years-old woman, hypertensive, hospitalized for a large cervical mass appeared 30 years ago.',
 'entities': "[{'id': '6759', 'offsets': array([22, 34]), 'role': '', 'semantic_type_id': '', 'text': 'hypertensive', 'type': 'EVENT'}\n {'id': '6774', 'offsets': array([36, 48]), 'role': '', 'semantic_type_id': '', 'text': 'hospitalized', 'type': 'EVENT'}\n {'id': '7666', 'offsets': array([61, 74]), 'role': '', 'semantic_type_id': 'C0149736', 'text': 'cervical mass', 'type': 'CLINENTITY'}\n {'id': '7820', 'offsets': array([55, 74]), 'role': '', 'semantic_type_id': '', 'text': 'large cervical mass', 'type': 'BODYPART'}\n {'id': '7988', 'offsets': array([ 0, 20]), 'role': 'PATIENT', 'semantic_type_id': '', 'text': 'A 50-years-old woman', 'type': 'ACTOR'}\n {'id': '8071', 'offsets': array([84, 92]), 'role': '', 'semantic_type_id': '', 'text': '30 years', 'type': 'TIMEX3'}]",
 'original_text': 'A 50-years-old woman, hypertensive, hospitalized for a large cervical mass appeared

In [2]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 100].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal_threshold,f1_score,precision,recall


In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = [100]
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/noInstr_8bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
output_cleaner = OutputCleaner(verbose=False)

print(evaluation_results)
for file in csv_files:
    if file.strip().endswith('0.0008.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal_threshold in similar_is_equal_threshold_list:
        # print(f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}")
        evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
        try:
            evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                                words_level=True, 
                                                similarity_types=['case', 'stop_words', 'subset', 'superset'])
            evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
        except Exception as e:
            print(e)
        #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
        # print('DONE')

In [10]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 100].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal_threshold,f1_score,precision,recall
7,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.649890,0.602338,0.705594
1,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.639212,0.634080,0.644426
6,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.638999,0.583906,0.705571
5,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.635105,0.597381,0.677914
13,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.626520,0.596308,0.659957
8,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.580823,0.556810,0.607000
16,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.574688,0.593377,0.557141
15,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.571382,0.595527,0.549118
2,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.559085,0.555629,0.562583
4,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,100,0.552100,0.578281,0.528186


### LLAMA 13B 4bit FT

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing_params_llama as postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = [100]
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/13B_4bit_FT/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
output_cleaner = OutputCleaner(verbose=False)

print(evaluation_results)
for file in csv_files:
    if file.strip().endswith('0.0008.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal_threshold in similar_is_equal_threshold_list:
        # print(f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}")
        evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
        evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                            words_level=True, 
                                            similarity_types=['case', 'stop_words', 'subset', 'superset'])
        #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
        evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
        # print('DONE')

In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 100].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal_threshold,f1_score,precision,recall
45,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.701161,0.746585,0.660948
9,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.699127,0.742332,0.660673
26,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.696993,0.726827,0.669513
46,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.695955,0.746409,0.651890
10,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.694581,0.732376,0.660495
29,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.689118,0.717028,0.663300
40,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.685441,0.699376,0.672050
14,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.681304,0.688892,0.673880
43,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.680618,0.685274,0.676024
42,data/llama/13B_4bit_FT/maxNewTokensFactor8_nSh...,100,0.680400,0.689366,0.671664


In [ ]:
import glob
import pandas as pd
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                                        words_level=True, 
                                                        similarity_types=['case', 'stop_words', 'subset', 'superset'])#,'leveshtein'])
                    # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [4]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 60].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
482,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,70,0.635337,0.614908,0.657171
480,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,80,0.635337,0.614908,0.657171
478,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,90,0.635337,0.614908,0.657171
485,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,100,0.635337,0.614908,0.657171
484,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,60,0.635337,0.614908,0.657171
...,...,...,...,...,...,...
598,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.008748,0.033635,0.005028
599,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,70,0.008748,0.033635,0.005028
600,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,65,0.008748,0.033635,0.005028
601,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,60,0.008748,0.033635,0.005028


In [15]:
## evaluation_results.to_csv('data/mistral/evaluation_results/results.csv', index=False)

In [23]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 100].sort_values(by='f1_score', ascending=False)['file'].to_list()[0:5]

['data/test_data_processed/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_8_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_8_0.0002.csv']

### One Run

In [10]:
eval_data[10]

{'sentence': 'Its surface was embossed and covered by a thin normal skin.',
 'entities': "[{'id': '6969', 'offsets': array([16, 24]), 'role': '', 'semantic_type_id': '', 'text': 'embossed', 'type': 'EVENT'}\n {'id': '6984', 'offsets': array([29, 36]), 'role': '', 'semantic_type_id': '', 'text': 'covered', 'type': 'EVENT'}\n {'id': '7850', 'offsets': array([40, 58]), 'role': '', 'semantic_type_id': '', 'text': 'a thin normal skin', 'type': 'BODYPART'}]",
 'original_text': 'A 50-years-old woman, hypertensive, hospitalized for a large cervical mass appeared 30 years ago. In the family history, her mother, sisters and cousins underwent a surgery for MNG. Despite of the large volume of the mass, the patient never described signs of cervical compression whatsoever respiratory, digestive, laryngeal, vascular or neurologic signs. She never suffered from thyroid dysfunction. Her neck was deformed by the voluminous formation classified grade III according to the WHO modified classification. The 

In [7]:
file ='/home/pferrazzi/mistral_finetuning/data/TMP_maxNewTokensFactor6_nShotsInference0_Mistral-7B-Instruct-v0.2_adapters_en.layer1.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=False, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 721.29it/s]
Generating train split: 681 examples [00:00, 21108.68 examples/s]
Map: 100%|██████████| 681/681 [00:00<00:00, 6140.25 examples/s]


JSONDecodeError: Extra data: line 1 column 281 (char 280)

In [10]:
cleaned_data[0]

{'sentence': 'A 46-year-old man with hypertension and dyslipidemia diagnosed 4-months before, as well as new-onset diabetes mellitus unveiled 1-month earlier, was referred to emergency department for hypokalemia.',
 'entities': "[{'id': '1614', 'offsets': array([23, 35]), 'role': '', 'semantic_type_id': '', 'text': 'hypertension', 'type': 'EVENT'}\n {'id': '1629', 'offsets': array([40, 52]), 'role': '', 'semantic_type_id': '', 'text': 'dyslipidemia', 'type': 'EVENT'}\n {'id': '1644', 'offsets': array([53, 62]), 'role': '', 'semantic_type_id': '', 'text': 'diagnosed', 'type': 'EVENT'}\n {'id': '1659', 'offsets': array([110, 118]), 'role': '', 'semantic_type_id': '', 'text': 'mellitus', 'type': 'EVENT'}\n {'id': '1674', 'offsets': array([149, 157]), 'role': '', 'semantic_type_id': '', 'text': 'referred', 'type': 'EVENT'}\n {'id': '1689', 'offsets': array([186, 197]), 'role': '', 'semantic_type_id': '', 'text': 'hypokalemia', 'type': 'EVENT'}\n {'id': '1996', 'offsets': array([ 91, 118]),

In [1]:
file ='data/TMP_maxNewTokensFactor6_nShotsInference0_Mistral-7B-Instruct-v0.2_adapters_en.layer1.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=True, similarity_types=['case', 'stop_words', 'subset', 'superset'])


NameError: name 'Dataset' is not defined

### BASE MODEL MISTRAL 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit_base/*.csv') 
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [2]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
133,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,80,0.333916,0.260224,0.465835
132,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,85,0.321196,0.250189,0.448482
131,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,90,0.312688,0.243325,0.437364
130,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,95,0.309780,0.241062,0.433297
138,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,100,0.308811,0.240308,0.431941
...,...,...,...,...,...,...
19,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,False,100,0.128506,0.281651,0.083243
31,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,90,0.127428,0.085827,0.247289
30,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,95,0.127009,0.085545,0.246475
38,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,100,0.126729,0.085357,0.245933


### BASE MODEL MISTRAL 4bit

In [2]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

evaluators = {}
csv_files = glob.glob('data/mistral/4bit_base/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    if not file.strip().endswith('BaseModel.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference4_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference3_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference1_BaseModel.csv


In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
13,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.466546,0.445486,0.489696
23,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.463026,0.446798,0.480477
33,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.461848,0.456885,0.466920
22,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.451200,0.434859,0.468818
12,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.446995,0.426217,0.469902
32,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.445845,0.440880,0.450922
21,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,90,0.442473,0.426345,0.459870
43,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.441708,0.445856,0.437636
20,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,95,0.440010,0.423869,0.457430
28,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,100,0.438910,0.422758,0.456345


In [4]:
##evaluation_results.to_csv('data/evaluation_results/mistral_4bit_base.csv', index=False)

### MISTRAL BASE 8bit

In [5]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

evaluators = {}
csv_files = glob.glob('data/mistral/8bit_base/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    if not file.strip().endswith('BaseModel.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor2_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor2_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference1_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference2_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference3_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference3_BaseMo

In [6]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
133,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,80,0.333916,0.260224,0.465835
132,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,85,0.321196,0.250189,0.448482
131,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,90,0.312688,0.243325,0.437364
130,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,95,0.309780,0.241062,0.433297
138,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,100,0.308811,0.240308,0.431941
...,...,...,...,...,...,...
19,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,False,100,0.128506,0.281651,0.083243
31,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,90,0.127428,0.085827,0.247289
30,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,95,0.127009,0.085545,0.246475
38,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,100,0.126729,0.085357,0.245933


In [7]:
##evaluation_results.to_csv('data/evaluation_results/mistral_8bit_base.csv', index=False)

### MISTRAL 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    # if file.strip().endswith('0.002.csv'):
    #     continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break

In [8]:
##evaluation_results.to_csv('data/evaluation_results/mistral_8bit.csv', index=False)

In [5]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
535,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.648048,0.633049,0.663774
54,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.641816,0.602763,0.686280
555,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.638147,0.598386,0.683568
164,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.638144,0.631369,0.645065
534,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,80,0.631801,0.615860,0.648590
...,...,...,...,...,...,...
669,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,100,0.005189,0.019928,0.002983
663,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,85,0.005189,0.019928,0.002983
662,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,90,0.005189,0.019928,0.002983
661,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,95,0.005189,0.019928,0.002983


### LLAMA 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/8bit/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [8]:
evaluator.data[1]

{'sentence': 'Hormonal study and dynamic biochemical tests performed indicated ECS.',
 'entities': "[{'id': '1704', 'offsets': array([ 9, 14]), 'role': '', 'semantic_type_id': '', 'text': 'study', 'type': 'EVENT'}\n {'id': '1719', 'offsets': array([39, 44]), 'role': '', 'semantic_type_id': '', 'text': 'tests', 'type': 'EVENT'}\n {'id': '1734', 'offsets': array([55, 64]), 'role': '', 'semantic_type_id': '', 'text': 'indicated', 'type': 'EVENT'}\n {'id': '1749', 'offsets': array([65, 68]), 'role': '', 'semantic_type_id': '', 'text': 'ECS', 'type': 'EVENT'}]",
 'original_text': 'A 46-year-old man with hypertension and dyslipidemia diagnosed 4-months before, as well as new-onset diabetes mellitus unveiled 1-month earlier, was referred to emergency department for hypokalemia. Hormonal study and dynamic biochemical tests performed indicated ECS. Imaging and cytological findings pointed toward a likely primary right parotid malignancy with liver metastases. Somatostatin receptor scintigraphy 

In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
354,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.405388,0.277098,0.754881
104,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.404637,0.276725,0.752440
353,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,80,0.398314,0.272060,0.743221
374,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.396249,0.271486,0.733189
103,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,80,0.396191,0.270660,0.738883
...,...,...,...,...,...,...
272,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,85,0.250214,0.182912,0.395879
271,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,90,0.248097,0.181227,0.393167
270,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,95,0.247070,0.180477,0.391540
278,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,100,0.246536,0.180080,0.390727


In [4]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_8bit.csv', index=False)

### LLAM7 BASE 4bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/7B_4bit_base/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
144,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,75,0.271827,0.230436,0.331345
143,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,80,0.261285,0.221447,0.318601
142,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,85,0.254501,0.215631,0.310466
141,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,90,0.248140,0.210160,0.302874
140,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,95,0.247668,0.209744,0.302332
...,...,...,...,...,...,...
12,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,85,0.003781,0.466667,0.001898
13,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,80,0.003781,0.466667,0.001898
18,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,100,0.003781,0.466667,0.001898
10,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,95,0.003781,0.466667,0.001898


In [4]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_4bit_base.csv', index=False)

### LLAMA7 BASE 8bit

In [5]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/7B_8bit_base/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference3_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference1_BaseModel.csv


In [6]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
34,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,75,0.287719,0.244096,0.350325
4,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,75,0.285771,0.532939,0.195228
33,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,80,0.276906,0.234850,0.337310
3,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,80,0.271771,0.506282,0.185738
32,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,85,0.271473,0.230189,0.330803
31,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,90,0.265436,0.225052,0.323482
30,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,95,0.264457,0.224170,0.322397
38,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,100,0.263790,0.223605,0.321584
2,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,85,0.257732,0.479351,0.176247
1,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,90,0.252131,0.468681,0.172451


In [7]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_8bit_base.csv', index=False)

### MISTRAL BASE simplest_prompt one run

In [4]:
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

file = 'data/test_data_processed/maxNewTokensFactor6_nShotsInference0_Mistral-7B-v0.1_simplest_prompt_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 80
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator_mistral_base = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator_mistral_base.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold)


file = 'data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 80
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator_mistral_instruct = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator_mistral_instruct.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold)

print(evaluator_mistral_instruct.evaluation_table)
print(evaluator_mistral_base.evaluation_table)



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1050.15it/s]
Generating train split: 681 examples [00:00, 20161.65 examples/s]
Map: 100%|██████████| 681/681 [00:00<00:00, 6854.36 examples/s]


{'evaluation':      TP  FP  FN
0     0   0  10
1     0   1   4
2     0   0   4
3     5   0   0
4     1   0   1
..   ..  ..  ..
676   0   0   4
677   3   0   1
678   3   0   0
679   3   1   2
680   0   0   4

[681 rows x 3 columns], 'precision': 0.7130730050933786, 'recall': 0.34164859002169196, 'f1': 0.46196150320806595}
{'evaluation':      TP  FP  FN
0     2   0   8
1     3   0   1
2     2   1   2
3     3   0   2
4     2   1   0
..   ..  ..  ..
676   2   1   2
677   3   0   1
678   2   1   1
679   2   1   3
680   3   0   1

[681 rows x 3 columns], 'precision': 0.6405353728489483, 'recall': 0.2725054229934924, 'f1': 0.3823473463952825}


In [6]:
eval_data[2]

{'sentence': 'Imaging and cytological findings pointed toward a likely primary right parotid malignancy with liver metastases.',
 'entities': "[{'id': '1764', 'offsets': array([24, 32]), 'role': '', 'semantic_type_id': '', 'text': 'findings', 'type': 'EVENT'}\n {'id': '1779', 'offsets': array([33, 40]), 'role': '', 'semantic_type_id': '', 'text': 'pointed', 'type': 'EVENT'}\n {'id': '2010', 'offsets': array([57, 89]), 'role': '', 'semantic_type_id': 'C1306459', 'text': 'primary right parotid malignancy', 'type': 'CLINENTITY'}\n {'id': '2017', 'offsets': array([ 95, 111]), 'role': '', 'semantic_type_id': 'C0494165', 'text': 'liver metastases', 'type': 'CLINENTITY'}\n {'id': '2052', 'offsets': array([57, 89]), 'role': '', 'semantic_type_id': '', 'text': 'primary right parotid malignancy', 'type': 'BODYPART'}\n {'id': '2058', 'offsets': array([ 95, 111]), 'role': '', 'semantic_type_id': '', 'text': 'liver metastases', 'type': 'BODYPART'}]",
 'original_text': 'A 46-year-old man with hypert

In [ ]:
file ='data/llama/7B_8bit/maxNewTokensFactor8_nShotsInference2_Llama-2-7b-chat-hf_adapters_en.layer1_8_torch.bfloat16_32_32_0.01_2_0.0002.csv'
eval_data = Dataset.from_csv(file) 

output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))
cleaned_data[3]
evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=True, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])

### MISTRAL 4bit

In [ ]:
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
# from utils.output_cleaner import OutputCleaner

file ='data/mistral/noInstr_8bit/maxNewTokensFactor8_nShotsInference0_Mistral-7B-v0.1_simplest_prompt_adapters_en.layer1_8_torch.bfloat16_16_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=False, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])


In [ ]:
from evaluation import evaluate


res = evaluate('data/mistral/noInstr_8bit',
         'data/evaluation_results/mistral_noInstr_8bit.csv',
         words_level=True,
         similar_is_equal_threshold_list=[80, 100],
         similarity_types=['case', 'stop_words', 'subset', 'superset'],
         wrong_keys_to_entity=False,
         offset=False)


### LLAMA 13B 4bit

In [3]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
# from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = [100]
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/13B_4bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
output_cleaner = OutputCleaner(verbose=False)

print(evaluation_results)
for file in csv_files:
    if file.strip().endswith('0.0008.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal_threshold in similar_is_equal_threshold_list:
        # print(f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}")
        evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
        evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                            words_level=True, 
                                            similarity_types=['case', 'stop_words', 'subset', 'superset'])
        #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
        evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
        # print('DONE')

Empty DataFrame
Columns: [file, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_4_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference4_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_8_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference4_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_2_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_4_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_2_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_8_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewToke

In [5]:
## evaluation_results.to_csv('data/evaluation_results/llama13B_4bit_wordsLevelTrue_evaluation.csv', index=False)

In [4]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal_threshold,f1_score,precision,recall
51,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.708628,0.752786,0.669364
8,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.704166,0.748645,0.664676
11,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.695741,0.736405,0.659334
27,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.693492,0.725360,0.664306
28,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.691360,0.721092,0.663983
44,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.690166,0.699494,0.681084
40,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.689516,0.695189,0.683935
21,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.687684,0.700227,0.675583
43,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.685843,0.687754,0.683942
52,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,100,0.682525,0.739965,0.633361


In [20]:
for i in range(5):
    print(evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False).iloc[i]['file'])

data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv
data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv
data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv
data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv
data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference0_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_2_0.0002.csv


In [ ]:
{'data/mistral/8bit_base': IndexError('list index out of range')}, 
{'data/mistral/8bit': IndexError('list index out of range')}, 
{'data/qwen/7B_8bit_base': JSONDecodeError('Extra data: line 1 column 281 (char 280)')}, 
{'data/qwen/14B_4bit_base': JSONDecodeError('Extra data: line 1 column 281 (char 280)')}, 
{'data/qwen/14B_8bit_base': JSONDecodeError('Extra data: line 1 column 281 (char 280)')}, 
{'data/qwen/7B_4bit_base': JSONDecodeError('Extra data: line 1 column 281 (char 280)')}]

In [17]:
import glob
import pandas as pd
from datasets import Dataset
from utils.evaluator import Evaluator
# from utils.output_cleaner import OutputCleaner

similar_is_equal_threshold_list = [100]
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/13B_NoQuantbit_base/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
output_cleaner = OutputCleaner(verbose=True)

print(evaluation_results)
for file in csv_files:
    if file.strip().endswith('0.0008.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal_threshold in similar_is_equal_threshold_list:
        # print(f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}")
        evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
        evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                            words_level=True, 
                                            similarity_types=['case', 'stop_words', 'subset', 'superset'])
        #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
        evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
        # print('DONE')

Empty DataFrame
Columns: [file, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/llama/13B_NoQuantbit_base/maxNewTokensFactor2_nShotsInference4_BaseModel_Llama-2-13b-chat-hf_NoQuant.csv
ORIGINAL model_response:  [{'entity': 'man'}, {'entity': 'hypertension'}, {'entity': 'dyslipidemia'}]
ORIGINAL model_response:  [{'entity': 'hormonal'}, {'entity': 'study'}, {'entity': 'dynamic'}]
ORIGINAL model_response:  [{'entity': 'imaging'}, {'entity': 'cytological'}, {'entity': 'findings'}]
ORIGINAL model_response:  [{'entity': 'patient'}, {'entity': 'underwent'}, {'entity': 'right parotidectomy'}]
ORIGINAL model_response:  [{'entity': 'managed'}, {'entity': 'hypercortisolism'}, {'entity': 'metyrapone'}]
ORIGINAL model_response:  [{'entity': 'woman'}, {'entity': 'hypertensive'}, {'entity': 'hospitalized'}]
ORIGINAL model_response:  [{'entity': 'history'}, {'entity': 'mother'}, {'entity': 'sisters'}]
ORIGINAL model_response:  [{'entity': 'volume'}, {'entity': 'mass'}, 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
import json
import re
from typing import Tuple
from typing import List

class OutputCleaner():
    def __init__(self, verbose=False) -> None:
        self.verbose = verbose
  
    def _remove_space_from_dict_keys(self, model_ouput_list: list) -> list:
        """
        Remove the spaces from the keys of a dictionary. E.g., [{"entity ": "value"}] -> [{"entity": "value"}]

        Args:
        model_ouput_list (dict): the list of dictionaries to be cleaned

        return:
        list: the cleaned list of dicts
        """
        out = []
        for dict in model_ouput_list:
            out.append({k.replace(' ', ''): v for k, v in dict.items()})
        return out
    
    def _drop_duplicates(self, model_response: list) -> str:
        """
        Drop the duplicates from a list. This is useful when the model output contains the same entity multiple times.

        Args:
        model_response (str): the model response with no duplicates
        """
        try :
            return list({v['entity']:v for v in model_response}.values())
        except Exception as error:
            model_response = self._remove_space_from_dict_keys(model_response)
            return list({v['entity']:v for v in model_response}.values())
        
    def _assess_model_output(self, model_response: str) -> bool:
        """
        Check if the model output is in the right format. If not, return False.
        
        Args:
        model_output (str): the postprocessed model output after beeing passed to _postprocess_model_output()

        return:
        bool: True if the format is correct, False otherwise
        """
        good_format = True
        try :
            res = json.loads(model_response)
            # print( res)
        except:
            good_format = False
        return good_format

            
    def _remove_json_special_chars(self, string):
        # print('sto pulendo: ', string)
        chars = ['\xa0', '\x80', '\x93']
        for char in chars:
            string = string.replace(char, ' ')
        return string
    
    def _clean_ground_truth(self, example: dict) -> dict:
        ground_truth = example['ground_truth']
        # print('inner ground truth: ', ground_truth)
        ground_truth = self._remove_json_special_chars(ground_truth)
        if '</s>' in ground_truth:
            ground_truth = ground_truth.replace('</s>', '')
        if ground_truth.strip() == ']':
            ground_truth = '[]'
        # print('mid ground truth: ', ground_truth)
        return({'ground_truth': ground_truth})

    def _clean_model_output(self, example: dict,  wrong_keys_to_entity:bool, latest_version:bool=True) -> dict:
        """
        Postprocess the model output to return a json like formatted string that can be used to compute the F1 score.

        Args:
        model_output (str): the model output as it is returned by the model. The processing of the output is done in the function
        wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'. If not, all keys that are not 'entity' are dropped

        return:
        dict: the model response

        """
        def has_unclosed_square_brackets(s:str)  -> bool:
            count = 0
            for char in s:
                if char == '[':
                    count += 1
                elif char == ']':
                    count -= 1
                    if count < 0:
                        return True
            return count > 0
        
        def has_unopen_square_brackets(s:str)  -> bool:
            count = 0
            for char in s:
                if char == '[':
                    count -= 1
                elif char == ']':
                    count += 1
                    if count > 0:
                        return True
            return count > 0
        
        def is_empty_list(string:str)  -> bool:
            if string=='[]':
                return True
            return False
        
        def is_list_of_lists(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, list) for item in tmp):
                    return True
            return False
        
        def is_list_of_dicts(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    return True
            return False
        
        def is_list_of_lists_and_dict(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                found_dict = False
                found_list = False
                for element in tmp:
                    if isinstance(element, list):
                        found_list = True
                    elif isinstance(element, dict):
                        found_dict = True
                    if found_list and found_dict:
                        return True
            return False
        
        def is_list_of_strings(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, str) for item in tmp):
                    return True
            return False

        def is_list_of_empty_dict(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                print('TMP: ', tmp)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    if all(str(item) == "{}" for item in tmp):
                        return True
            return False

        def is_list_with_one_empty_dict(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and len(tmp) == 1 and isinstance(tmp[0], dict) and tmp[0] == {}:
                    return True
            return False
        
        def is_list_of_dict_numeric_values(string:str)  -> bool:
            #print('STRING: ', string)
            if self._assess_model_output(string):
                tmp = json.loads(string)
                #print('TMP: ', tmp)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    for item in tmp:
                        if len(item.values()) > 0:
                            val = list(item.values())[0] 
                            if isinstance(val, int) or isinstance(val, float):
                                return True
            return False
        
        def is_list_of_dicts_none_values(string:str) -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    for item in tmp:
                        if len(item.values()) > 0:
                            val = list(item.values())[0] 
                            if val is None:
                                return True
            return False

        def is_list_of_dicts_and_strings(string:str)  -> bool:
            if self._assess_model_output(string):
                #print('ASSESSED')
                tmp = json.loads(string)
                found_dict = False
                found_string = False
                for element in tmp:
                    if isinstance(element, str):
                        found_string = True
                    elif isinstance(element, dict):
                        found_dict = True
                    if found_string and found_dict:
                        return True
            return False
        
        def is_string(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, str):
                    return True
            return False
        
        def is_list_of_strings_representing_dicts(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, str) for item in tmp):
                    tmp_list = []
                    for item in tmp:
                        if self._assess_model_output(item):
                          tmp_list.append(json.loads(item))
                    if all(isinstance(item, dict) for item in tmp_list):
                        return True
            return False
        
        def is_list_of_dicts_of_lists(string:str)  -> bool:
            # print('STRING: ', string)
            if self._assess_model_output(string):
                tmp = json.loads(string)
                # print('TMP: ', tmp)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    for item in tmp:
                        # print('item: ',item)
                        tmp2 = list(item.values())[0]
                        if len(tmp2) > 0:
                            if isinstance(list(item.values())[0], list):
                                return True
            return False
        
        def is_numeric(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, (int, float)):
                    return True
            return False
        
        def are_entities_extracted_as_dict_keys_instead_of_values(string:str, example:dict) -> bool:
            if is_list_of_dicts(string):
                tmp = json.loads(string)
                keys = [key for item in tmp for key in item.keys()]
                if 'entity' not in keys:
                    if all(entity in example['sentence'] for entity in keys):
                        return True
            return False
        
        
        
        def convert_wrong_keys_into_entity(string:str) -> List[str]:
            if is_list_of_dicts(string):
                tmp = json.loads(string)
                tmp = [str({"entity":v}) for el in tmp for v in el.values()]
                return tmp
            else:
                return []


        def only_dicts_with_key_entity(string:str, wrong_keys_to_entity:bool) -> Tuple[bool, str]:
            """
            Extract only the dictionaries with the key 'entity' in the list of dictionaries in the string
            
            Args:
            string (str): the string to be cleaned
            wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'
            """
            els_between_curly = re.findall(r'\{(.+?)\}', string)
            clean = [el for el in els_between_curly if el.startswith('"entity"') or el.startswith("'entity'")]
            clean = ['{' + el + '}' for el in clean]
            dirty = []
            if wrong_keys_to_entity:
                dirty = [el for el in els_between_curly if (not el.startswith('"entity"')) and (not el.startswith("'entity'"))]
                dirty = ['{' + el + '}' for el in dirty]
                dirty = '[' + ', '.join(dirty) + ']'
                cleaned_dirty = convert_wrong_keys_into_entity(dirty)
                out = '[' + ', '.join(clean) + ', '.join(cleaned_dirty) +  ']'
            else:
                out = '[' + ', '.join(clean) + ']'
            out = out.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(" \'", " \"")
            operations_performed = False
            if len(clean) != len(els_between_curly):
                operations_performed = True
            if is_empty_list(out):
                return operations_performed, '[{"entity":""}]'
            return operations_performed, str(out)
        

        model_output = example['model_responses']   
        if self.verbose: print('ORIGINAL MODEL OUTPUT:  ', model_output)
        if self.verbose: print('GROUND TRUTH: ', example['ground_truth'])
        # model_output = self._exceptions_handler(model_output)
        
        if model_output is None or is_empty_list(model_output):
            return {'model_output':'[{"entity":""}]'}
        
        model_output = self._remove_json_special_chars(model_output)
        # print('PULITO: ', model_output)
                
        if are_entities_extracted_as_dict_keys_instead_of_values(model_output, example):
            # print('ENTITIES EXTRACTED AS DICT KEYS INSTEAD OF VALUES')
            tmp = json.loads(model_output)
            tmp = [{"entity":k} for el in tmp for k in el.keys() ]
            tmp = str(tmp)
            return {'model_output':tmp}

        if is_numeric(model_output):
            # print('IS NUMERIC')
            return {'model_output':'[{"entity":""}]'}

        
        if is_list_of_strings_representing_dicts(model_output):
            # print('is_list_of_strings_representing_dicts')                
            tmp = json.loads(model_output)
            tmp_list = []
            for item in tmp:
                if self._assess_model_output(item):
                  tmp_list.append(json.loads(item))
            return {'model_output':str(tmp_list)}
      
        
        if is_list_of_lists_and_dict(model_output):
            # print('is_list_of_lists_and_dict')
            tmp = json.loads(model_output)
            for el in tmp:
                if isinstance(el, list):
                    tmp = str(el)
                    # print('is_list_of_lists_and_dict')
                    return {'model_output':tmp}
                
        if is_list_of_lists(model_output):
            # print('is_list_of_lists')
            tmp = json.loads(model_output)
            tmp2 = str(tmp[0]).replace("'", "\"")
            if is_list_of_dicts_and_strings(tmp2):
                tmp = tmp[0]
                out = [item for item in tmp if isinstance(item, dict)]
                return {'model_output':str(out)} 
            tmp = str(tmp[0])
            return {'model_output':tmp}

        if is_list_of_strings(model_output):
            if self.verbose: print('is_list_of_strings')
            tmp = json.loads(model_output)
            tmp = [{"entity":el} for el in tmp]
            tmp = str(tmp)
            # print('is_list_of_strings')
            return {'model_output': tmp}
        
        if is_string(model_output):
            # model_output = model_output.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(" \'", " \"")
            # print('PULITO: ', model_output)
            tmp = json.loads(model_output)
            if all(el in tmp for el in ['{', 'entity', '}']):
                return {'model_output':tmp}
            tmp = [{"entity":tmp}]
            tmp = str(tmp)
            #print('is_string')
            return {'model_output':tmp}

        
        if latest_version:
            model_output = self._extract_text_between_curl_brackets(model_output)
            model_output = self._clean_text_between_curl_brackets(model_output)
                    
            if is_list_of_dict_numeric_values(model_output):
                #print('is_list_of_dict_int_values')
                tmp = json.loads(model_output)
                tmp = [str({"entity":str(v)}) for el in tmp for v in el.values()]
                model_output = str(tmp)
            
            if is_list_of_dicts_none_values(model_output):
                if self.verbose: print('is_list_of_dicts_none_values')
                tmp = json.loads(model_output)
                tmp = [str({"entity":v}) for el in tmp for v in el.values() if v is not None]
                model_output = str(tmp)
                    
            if is_list_of_empty_dict(model_output):
                # print('is_list_of_empty_dict')
                return {'model_output':'[{"entity":""}]'}
            
            if is_list_of_dicts_of_lists(model_output):
                if self.verbose: print('is_list_of_dicts_of_lists')
                tmp = json.loads(model_output)
                tmp = [{"entity":v} for el in tmp for v in el.values() if not isinstance(v, list)]
                return {'model_output':str(tmp)}  
                
            # print('CLEANED:  ', model_output)
            cleaning_done, cleaned_model_output = only_dicts_with_key_entity(model_output, wrong_keys_to_entity=wrong_keys_to_entity)
            if cleaning_done:
                model_output = cleaned_model_output
            
            if is_list_of_dicts(model_output):
                #print('PRE CLEANED: ', model_output)
                tmp = json.loads(model_output)
                return {'model_output':str(tmp)}
            
            else: 
                # print('NOT CLEANED: ', model_output, '\n\n')
                return {'model_output':'[{"entity":""}]'}
        
            
    def _exceptions_handler(self, model_output: str, error) -> str:
        # if hasattr(error, 'msg'):
        #     if error.msg.startswith('Expecting property name enclosed in double quotes'):
        #         model_output = model_output.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(": \'", ": \"")
        
        try:
            json.loads(model_output)
        except Exception as error:
            if isinstance(error, json.decoder.JSONDecodeError):
                #if error.msg == "Expecting ',' delimiter":
                key_part, value_part = model_output.split(': ', 1)
                first_occurrence = value_part.find('"')
                last_occurrence = value_part.rfind('"')
                model_output = key_part + ': "' + value_part[first_occurrence+1:last_occurrence].replace("'", r'\'') + '"' + '}'
        return model_output
    
    def _substitute_apexes(self, model_output: str) -> str:
        model_output = model_output.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(": \'", ": \"")
        return model_output
    
    
    def _extract_text_between_curl_brackets(self, model_output: str) -> str:
        """
        Extract the text between the curl brackets of the model output, as enities are usually outputted in this format: {"entity": "value"}

        Args:
        model_output (str): the example from the dataset

        """
        text_between_curl_brackets = re.findall(r'\{(.+?)\}', model_output)
        cleaned_output = ['{'+ el +'}' for el in text_between_curl_brackets]
        cleaned_output = '[' + ', '.join(cleaned_output) + ']'
        return cleaned_output
    

    def _clean_text_between_curl_brackets(self, text_between_curl_brackets: str) -> str:
        """
        Clean the text between the curl brackets of the model output, as entities are usually outputted in this format: {"key": "value"}

        Args:
        model_output (str): the example from the dataset

        """
        text_between_curl_brackets = re.sub(r'",(.+?)}', r'"}', text_between_curl_brackets)
        return text_between_curl_brackets

    # def _keep_only_one_keyVal_from_dict(dict: dict) -> dict:
    #     """
    #     Keep only one key-value pair from a dictionary. This is useful when the model outputs a dictionary with more than one key-value pair.

    #     Args:
    #     dict (dict): the dictionary to be cleaned

    #     return:
    #     dict: the cleaned dictionary
    #     """
    #     k_first = "entity"
    #     if k_first not in dict.keys():
    #         k_first = list(dict.keys())[0]
    #     return {k_first:dict[k_first]}

    
    def apply_cleaning(self, data, wrong_keys_to_entity) -> None:
        """
        Apply the cleaning to the model output and return the cleaned response in a new cloumn called 'model_output

        Args:
        model_output (str): the model output as it is returned by the model. The processing of the output is done in the function
        wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'. If not, all keys that are not 'entity' are dropped

        return:
        str: the model response, i.e. the model output without the instruction
        """
        print('sAPPLY CLEANING')
        data = data.map(lambda x: self._clean_ground_truth(x), remove_columns=['ground_truth'])
        data = data.map(lambda x: self._clean_model_output(x, wrong_keys_to_entity)) 
        return data

In [22]:
from datasets import Dataset
from utils.evaluator import Evaluator
#from utils.output_cleaner import OutputCleaner

file ='data/mistral/8bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_8_torch.bfloat16_32_32_0.01_8_0.0008.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner(verbose=True)
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=True, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])

sAPPLY CLEANING


Map:   0%|          | 0/681 [00:00<?, ? examples/s]

Map:  49%|████▉     | 335/681 [00:00<00:00, 4706.64 examples/s]

ORIGINAL MODEL OUTPUT:   ["entity": "hypertension"}, {"entity": "dyslipidemia"}, {"entity": "diabetes"}, {"entity": "unveiled"}, {"entity": "referred"}, {"entity": "hypokalemia"}, {"entity": "dyslipidemia"}, {"entity": "new-onset diabetes mellitus"}, {"entity": "A 46-year-old man"}, {"entity": "4-months before"}, {"entity": "1-month earlier"
GROUND TRUTH:  [{"entity": "hypertension"}, {"entity": "dyslipidemia"}, {"entity": "diagnosed"}, {"entity": "mellitus"}, {"entity": "referred"}, {"entity": "hypokalemia"}, {"entity": "new-onset diabetes mellitus"}, {"entity": "A 46-year-old man"}, {"entity": "4-months"}, {"entity": "1-month"}]
TMP:  [{'entity': 'dyslipidemia'}, {'entity': 'diabetes'}, {'entity': 'unveiled'}, {'entity': 'referred'}, {'entity': 'hypokalemia'}, {'entity': 'dyslipidemia'}, {'entity': 'new-onset diabetes mellitus'}, {'entity': 'A 46-year-old man'}, {'entity': '4-months before'}]
ORIGINAL MODEL OUTPUT:   ], {"entity": "study"}, {"entity": "biochemical tests"}, {"entity":

IndexError: list index out of range

In [4]:
eval_data[0]

{'sentence': 'A 46-year-old man with hypertension and dyslipidemia diagnosed 4-months before, as well as new-onset diabetes mellitus unveiled 1-month earlier, was referred to emergency department for hypokalemia.',
 'entities': "[{'id': '1614', 'offsets': array([23, 35]), 'role': '', 'semantic_type_id': '', 'text': 'hypertension', 'type': 'EVENT'}\n {'id': '1629', 'offsets': array([40, 52]), 'role': '', 'semantic_type_id': '', 'text': 'dyslipidemia', 'type': 'EVENT'}\n {'id': '1644', 'offsets': array([53, 62]), 'role': '', 'semantic_type_id': '', 'text': 'diagnosed', 'type': 'EVENT'}\n {'id': '1659', 'offsets': array([110, 118]), 'role': '', 'semantic_type_id': '', 'text': 'mellitus', 'type': 'EVENT'}\n {'id': '1674', 'offsets': array([149, 157]), 'role': '', 'semantic_type_id': '', 'text': 'referred', 'type': 'EVENT'}\n {'id': '1689', 'offsets': array([186, 197]), 'role': '', 'semantic_type_id': '', 'text': 'hypokalemia', 'type': 'EVENT'}\n {'id': '1996', 'offsets': array([ 91, 118]),

In [3]:
cleaned_data[23]

NameError: name 'cleaned_data' is not defined

In [6]:
from datasets import Dataset
from utils.evaluator import Evaluator
# from utils.output_cleaner import OutputCleaner

file ='data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference3_BaseModel.csv'
eval_data = Dataset.from_csv(file) 
eval_data[1]    

{'sentence': 'Hormonal study and dynamic biochemical tests performed indicated ECS.',
 'entities': "[{'id': '1704', 'offsets': array([ 9, 14]), 'role': '', 'semantic_type_id': '', 'text': 'study', 'type': 'EVENT'}\n {'id': '1719', 'offsets': array([39, 44]), 'role': '', 'semantic_type_id': '', 'text': 'tests', 'type': 'EVENT'}\n {'id': '1734', 'offsets': array([55, 64]), 'role': '', 'semantic_type_id': '', 'text': 'indicated', 'type': 'EVENT'}\n {'id': '1749', 'offsets': array([65, 68]), 'role': '', 'semantic_type_id': '', 'text': 'ECS', 'type': 'EVENT'}]",
 'original_text': 'A 46-year-old man with hypertension and dyslipidemia diagnosed 4-months before, as well as new-onset diabetes mellitus unveiled 1-month earlier, was referred to emergency department for hypokalemia. Hormonal study and dynamic biochemical tests performed indicated ECS. Imaging and cytological findings pointed toward a likely primary right parotid malignancy with liver metastases. Somatostatin receptor scintigraphy 